In [1]:
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer


Считываем данные

In [2]:
df = pd.read_csv('pricerunner_aggregate.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
data = df[['product_title', 'merchant_id', 'cluster_label','category_id']]

Обучаем модель catboost

In [35]:
text_features = ['product_title', 'cluster_label']

# Замените 'целевая_переменная' на название вашей целевой переменной
X = data.drop('category_id', axis=1)
y = data['category_id']

# Разделение на тренировочный и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание объекта Pool с текстовыми признаками
train_pool = Pool(data=X_train, label=y_train, text_features=text_features)
test_pool = Pool(data=X_test, label=y_test, text_features=text_features)

# Создание модели CatBoost
model_cat_boost = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.05, loss_function='MultiClass', random_seed=42)

# Обучение модели
model_cat_boost.fit(train_pool, eval_set=test_pool, verbose=50)

# Оценка качества модели на тестовом наборе
accuracy = model_cat_boost.score(test_pool)
print(f'Test Accuracy: {accuracy}')

0:	learn: 1.9330453	test: 1.9299142	best: 1.9299142 (0)	total: 15.7s	remaining: 2h 10m 30s
50:	learn: 0.2028315	test: 0.1901714	best: 0.1901714 (50)	total: 12m 57s	remaining: 1h 54m 7s
100:	learn: 0.1147309	test: 0.1101140	best: 0.1101140 (100)	total: 26m 14s	remaining: 1h 43m 41s
150:	learn: 0.0934068	test: 0.0922798	best: 0.0922798 (150)	total: 39m 58s	remaining: 1h 32m 24s
200:	learn: 0.0831598	test: 0.0849152	best: 0.0849152 (200)	total: 54m 1s	remaining: 1h 20m 21s
250:	learn: 0.0758888	test: 0.0802585	best: 0.0802585 (250)	total: 1h 7m 56s	remaining: 1h 7m 23s
300:	learn: 0.0705182	test: 0.0771310	best: 0.0771310 (300)	total: 1h 22m 33s	remaining: 54m 35s
350:	learn: 0.0660740	test: 0.0751258	best: 0.0751258 (350)	total: 1h 37m 20s	remaining: 41m 19s
400:	learn: 0.0621390	test: 0.0733597	best: 0.0733597 (400)	total: 1h 51m 18s	remaining: 27m 28s
450:	learn: 0.0590970	test: 0.0721568	best: 0.0721568 (450)	total: 2h 5m 22s	remaining: 13m 37s
499:	learn: 0.0567450	test: 0.0713714	be

Обучаем логистическую регрессию 

In [23]:
X = df[['merchant_id','cluster_id']]
y = df['category_id']

# Разделение на тренировочный и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_log_reg = LogisticRegression()
model_log_reg.fit(X_train_scaled, y_train)
y_pred = model_log_reg.predict(X_test_scaled)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7969701260087781


/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
X = df[['product_title','cluster_label']]
y = df['category_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()

# Инициализация и обучение TfidfVectorizer для каждой из текстовых колонок
tfidf_vectorizer_text1 = TfidfVectorizer()
X_train_tfidf_text1 = tfidf_vectorizer_text1.fit_transform(X_train['product_title'])
X_test_tfidf_text1 = tfidf_vectorizer_text1.transform(X_test['product_title'])

tfidf_vectorizer_text2 = TfidfVectorizer()
X_train_tfidf_text2 = tfidf_vectorizer_text2.fit_transform(X_train['cluster_label'])
X_test_tfidf_text2 = tfidf_vectorizer_text2.transform(X_test['cluster_label'])

# Объединение TF-IDF матриц для каждой из колонок
X_train_tfidf_combined = pd.concat([pd.DataFrame(X_train_tfidf_text1.toarray()), pd.DataFrame(X_train_tfidf_text2.toarray())], axis=1)
X_test_tfidf_combined = pd.concat([pd.DataFrame(X_test_tfidf_text1.toarray()), pd.DataFrame(X_test_tfidf_text2.toarray())], axis=1)

# Создание и обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train_tfidf_combined, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test_tfidf_combined)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9736655811977913
